In [27]:
import pandas as pd
import csv

In [28]:
TOTAL_COST_SUM = 0 # For (temporary) A computation (obtained below)
TOTAL_NUM_FRAGMENTS = 243  # For (temporary) B computation

In [29]:
def to_array(s):
    s = s.replace(r'{', '')
    s = s.replace(r'}', '')
    s = s.replace(r' ', '')    

    if s == '': return []

    # return list(map(int,s.split(',')))

    return s.split(',')

In [30]:
original_df = pd.read_csv("Statement-Infomation.csv")
original_df

,Procedure,Statement,producer statement,consumer statement,cost,result size,Fragment
0,5,25,{},"{29, 30, 32}",0.061239,99090,"{88,95,96,97,101,102,103,87}"
1,5,26,{},"{29, 30, 32}",0.045698,73049,"{30,24}"
2,5,27,{},{29},0.047481,2880404,"{18,17,15,16,1,4}"
3,5,28,{},{30},0.064059,1441548,"{150,149,147,148,125,128}"
4,5,29,"{25,26,27}",{33},0.174254,188834,{}
5,5,30,"{25,26,28}",{33},0.169058,135789,{}
6,5,31,{},{32},0.063730,719384,"{186,211,210,208,209,190}"
7,5,32,"{25,26,31}",{33},0.171958,56139,{}
8,5,33,"{29,30,32}",{34},0.252135,380762,{}
9,5,34,{33},{35},1.182939,555294969,{}


In [31]:
original = {}

for i in range(len(original_df)):
    row = original_df.loc[i]

    # NOTE: Procedure is skipped in this algorithm
    tmp = []
    tmp.append(to_array(row[2])) 
    tmp.append(to_array(row[3]))
    tmp.append(row[4])
    tmp.append(row[5])

    fragments = to_array(row[6])
    tmp.append(fragments)
    
    original[str(row[1])] = tmp

    # d = dict([(row[1], tmp)])
    # original.update(d)

    TOTAL_COST_SUM += row[4]
    TOTAL_NUM_FRAGMENTS += len(fragments)

In [32]:
original['25']

[[],
 ['29', '30', '32'],
 0.061239,
 99090,
 ['88', '95', '96', '97', '101', '102', '103', '87']]

In [33]:
def rule_A(cost: int) -> bool:
    if cost > TOTAL_COST_SUM/4: return False    # type: ignore
    else: return True

def rule_B(fragments: list) -> bool:
    if len(fragments) > TOTAL_NUM_FRAGMENTS/4: return False
    else: return True

In [34]:
def compute_total_benefit(info: dict):
    total_benefit = 0
    for statement in info:
        row = info[statement]

        if len(row) > 5:
            total_benefit += row[6]
    
    return total_benefit

In [35]:
def merge(info: dict, producer_key: str, consumer_key: str):
    '''
    Merge producer and consumer.
    '''
    p = info[producer_key]
    c = info[consumer_key]


    # Compute part of merged info for determination
    cost = p[2] + c[2]
    fragments = p[4] + c[4]


    # Determine whether to merge using rules.
    if not(rule_A(cost) and rule_B(fragments)): return None, None


    # Compute rest of merged info
    producer_statements = p[0] + c[0]
    producer_statements.remove(producer_key)    
    if producer_statements is None: producer_statements = []

    consumer_statements = p[1] + c[1]
    consumer_statements.remove(consumer_key)
    if consumer_statements is None: consumer_statements = []

    # Result size is computed to have the smallest result size.
    # if p[3] > c[3]: result_size = c[3]
    # else: result_size = p[3]

    result_size = c[3]  #p[3] + c[3]
    # If producer statement is merged one, then keep it as well.
    if len(p) > 5:  
        result_size += p[3]

    consist = []
    if len(p) > 5:  # == p is merged statement
        consist.extend(p[5])
    else: consist.append(producer_key)
    
    if len(c) > 5: consist.extend(c[5])
    else: consist.append(consumer_key)

    consist = sorted(consist)

    benefit = p[3]

    # Pack the result
    merged = [
        producer_statements,
        consumer_statements,
        cost,
        result_size,
        fragments,
        consist,
        benefit
    ]


    # Make new key
    key = "EB-" + "-".join(map(str, list(consist)))

    return key, merged

In [36]:
def update(info: dict, merged_keys: list[str], merge_key: str):
    '''
    Inplace function
    '''

    for merged_key in merged_keys:
        del info[merged_key]

    for statement in info:
        row = info[statement]

        def search_and_delete(array: list):
            deleted = False
            for i in reversed(range(len(array))):
                if array[i] in merged_keys:
                    del array[i]
                    deleted = True
                    
            if deleted: array.append(merge_key)
        
        search_and_delete(row[0])
        search_and_delete(row[1])

In [37]:
from copy import deepcopy

global result
result = []

def search(info: dict, history: list):
    for statement in info:
        # NOTE: statement is consumer
        row = info[statement]

        if len(row[0]) == 0: continue   # When have no producer statements
        
        for producer_statement in row[0]:
            producer_statement = str(producer_statement)
            k, m = merge(info, producer_statement, statement)
            # print(k, m)

            # When satisfied the rule
            if k is not None:
                # Update info
                updated_info = deepcopy(info)

                updated_info[k] = m
                update(updated_info, [producer_statement, statement], k)
                # print(updated_info)


                # Write on history
                tmp = {}
                tmp["update"] = k
                tmp["result"] = updated_info
                tmp["total_benefit"] = compute_total_benefit(updated_info)
                tmp["next"] = []
                history.append(tmp)
                
                # Recursively proceed search
                search(updated_info, tmp["next"])

search(original, result)

In [38]:
global final_result
final_result = []

def extract_terminal_node(tree: dict):
    if len(tree['next']) == 0:
        final_result.append(tree['result'])
    
    for subtree in tree['next']:
        extract_terminal_node(subtree)

for r in result:
    extract_terminal_node(r)

In [39]:
final_result[0]

{'34': [['EB-25-27-29-33'], ['35'], 1.182939, 555294969, []],
 '35': [['34'], [], 0.407706, 1, []],
 'EB-26-28-30-31-32': [['EB-25-27-29-33'],
  ['EB-25-27-29-33'],
  0.514503,
  191928,
  ['150',
   '149',
   '147',
   '148',
   '125',
   '128',
   '30',
   '24',
   '186',
   '211',
   '210',
   '208',
   '209',
   '190'],
  ['26', '28', '30', '31', '32'],
  135789],
 'EB-25-27-29-33': [['EB-26-28-30-31-32'],
  ['34', 'EB-26-28-30-31-32'],
  0.5351090000000001,
  569596,
  ['18',
   '17',
   '15',
   '16',
   '1',
   '4',
   '88',
   '95',
   '96',
   '97',
   '101',
   '102',
   '103',
   '87'],
  ['25', '27', '29', '33'],
  2880404]}

In [40]:
total_benefit_list = []

for i, f in enumerate(final_result):
    b = 0
    for st in f:
        if len(f[st]) > 5:
            b += f[st][6]
    total_benefit_list.append([i, b])

In [41]:
total_benefit_list_sorted = sorted(total_benefit_list, key=lambda x: -x[1])

In [42]:
best_benefit_list = []
for t in total_benefit_list_sorted:
    if t[1] != total_benefit_list_sorted[0][1]:
        break
    
    best_benefit_list.append(t)

In [43]:
len(best_benefit_list)

540

In [44]:
final_result[best_benefit_list[0][0]]

{'34': [['EB-25-26-27-29-33'], ['35'], 1.182939, 555294969, []],
 '35': [['34'], [], 0.407706, 1, []],
 'EB-28-30': [['EB-25-26-27-29-33'],
  ['EB-25-26-27-29-33'],
  0.23311700000000002,
  135789,
  ['150', '149', '147', '148', '125', '128'],
  ['28', '30'],
  1441548],
 'EB-31-32': [['EB-25-26-27-29-33'],
  ['EB-25-26-27-29-33'],
  0.235688,
  56139,
  ['186', '211', '210', '208', '209', '190'],
  ['31', '32'],
  719384],
 'EB-25-26-27-29-33': [['EB-28-30', 'EB-31-32'],
  ['EB-28-30', 'EB-31-32', 'EB-28-30', 'EB-31-32', '34'],
  0.580807,
  569596,
  ['18',
   '17',
   '15',
   '16',
   '1',
   '4',
   '30',
   '24',
   '88',
   '95',
   '96',
   '97',
   '101',
   '102',
   '103',
   '87'],
  ['25', '26', '27', '29', '33'],
  2880404]}

In [45]:
final_result[best_benefit_list[0][0]].keys(), final_result[best_benefit_list[1][0]].keys()

(dict_keys(['34', '35', 'EB-28-30', 'EB-31-32', 'EB-25-26-27-29-33']),
 dict_keys(['34', '35', 'EB-28-30', 'EB-31-32', 'EB-25-26-27-29-33']))

In [46]:
different_list = []

for best in best_benefit_list:
    if len(final_result[best[0]].keys()) != len(final_result[best_benefit_list[0][0]].keys()):
        print(best[0])

    is_different = False
    for i, key in enumerate(final_result[best[0]].keys()):
        if key != list(final_result[best_benefit_list[0][0]].keys())[i]:
            is_different = True
    
    if is_different: 
        different_list.append(best[0])

In [64]:
best_category = []

for best in best_benefit_list:
    statements = sorted(list(final_result[best[0]].keys()))

    def create_category():
        tmp = {}
        tmp["statements"] = statements
        tmp["index"] = []
        tmp["index"].append(best[0])

        best_category.append(tmp)

    if len(best_category) == 0:
        create_category()
        continue  
    
    is_matched_all = False
    for category in best_category:
        is_matched = True
        for i, j in zip(statements, category["statements"]):
            if i != j:                
                break
        
        if is_matched: 
            category["index"].append(best[0])
            is_matched_all = True
            break
    
    if not(is_matched_all):
        create_category()

In [66]:
best_category

[{'statements': ['34', '35', 'EB-25-26-27-29-33', 'EB-28-30', 'EB-31-32'],
  'index': [618,
   668,
   989,
   1004,
   1005,
   1107,
   1141,
   1142,
   2540,
   2590,
   2911,
   2926,
   2927,
   3029,
   3063,
   3064,
   4868,
   4877,
   4878,
   4932,
   4941,
   4942,
   4962,
   4963,
   4964,
   4965,
   4966,
   4967,
   5002,
   5030,
   5031,
   5087,
   5115,
   5116,
   5244,
   5245,
   5246,
   5247,
   5248,
   5249,
   6988,
   7038,
   7359,
   7374,
   7375,
   7477,
   7511,
   7512,
   8910,
   8960,
   9281,
   9296,
   9297,
   9399,
   9433,
   9434,
   11238,
   11247,
   11248,
   11302,
   11311,
   11312,
   11332,
   11333,
   11334,
   11335,
   11336,
   11337,
   11372,
   11400,
   11401,
   11457,
   11485,
   11486,
   11614,
   11615,
   11616,
   11617,
   11618,
   11619,
   12861,
   12932,
   13301,
   13316,
   13317,
   13503,
   13537,
   13538,
   13886,
   13957,
   14326,
   14341,
   14342,
   14528,
   14562,
   14563,
   14836,
   14

In [70]:
final_result[best_category[0]['index'][0]]

{'34': [['EB-25-26-27-29-33'], ['35'], 1.182939, 555294969, []],
 '35': [['34'], [], 0.407706, 1, []],
 'EB-28-30': [['EB-25-26-27-29-33'],
  ['EB-25-26-27-29-33'],
  0.23311700000000002,
  135789,
  ['150', '149', '147', '148', '125', '128'],
  ['28', '30'],
  1441548],
 'EB-31-32': [['EB-25-26-27-29-33'],
  ['EB-25-26-27-29-33'],
  0.235688,
  56139,
  ['186', '211', '210', '208', '209', '190'],
  ['31', '32'],
  719384],
 'EB-25-26-27-29-33': [['EB-28-30', 'EB-31-32'],
  ['EB-28-30', 'EB-31-32', 'EB-28-30', 'EB-31-32', '34'],
  0.580807,
  569596,
  ['18',
   '17',
   '15',
   '16',
   '1',
   '4',
   '30',
   '24',
   '88',
   '95',
   '96',
   '97',
   '101',
   '102',
   '103',
   '87'],
  ['25', '26', '27', '29', '33'],
  2880404]}

In [71]:
for i, category in enumerate(best_category):
    procedure = 5   

    with open(f"output/{i}.csv", "w") as c:
        writer = csv.writer(c)
        writer.writerow(list(original_df.columns) + ["consist", "benefit"])

        statements = final_result[category['index'][0]]
        for s in statements:
            row = []
            for col in statements[s]:
                if type(col) == list:
                    if len(col) == 0: col = r"{}"
                    else: 
                        col = r"{" + " ,".join(col) + r"}"
                row.append(col)
            row.insert(0, 5)
            row.insert(1, s)
            print(row)
            writer.writerow(row)

[5, '34', '{EB-25-26-27-29-33}', '{35}', 1.182939, 555294969, '{}']
[5, '35', '{34}', '{}', 0.407706, 1, '{}']
[5, 'EB-28-30', '{EB-25-26-27-29-33}', '{EB-25-26-27-29-33}', 0.23311700000000002, 135789, '{150 ,149 ,147 ,148 ,125 ,128}', '{28 ,30}', 1441548]
[5, 'EB-31-32', '{EB-25-26-27-29-33}', '{EB-25-26-27-29-33}', 0.235688, 56139, '{186 ,211 ,210 ,208 ,209 ,190}', '{31 ,32}', 719384]
[5, 'EB-25-26-27-29-33', '{EB-28-30 ,EB-31-32}', '{EB-28-30 ,EB-31-32 ,EB-28-30 ,EB-31-32 ,34}', 0.580807, 569596, '{18 ,17 ,15 ,16 ,1 ,4 ,30 ,24 ,88 ,95 ,96 ,97 ,101 ,102 ,103 ,87}', '{25 ,26 ,27 ,29 ,33}', 2880404]


In [51]:
final_result[best_category[0]['index'][0]]

{'34': [['EB-25-26-27-29-33'], ['35'], 1.182939, 555294969, []],
 '35': [['34'], [], 0.407706, 1, []],
 'EB-28-30': [['EB-25-26-27-29-33'],
  ['EB-25-26-27-29-33'],
  0.23311700000000002,
  135789,
  ['150', '149', '147', '148', '125', '128'],
  ['28', '30'],
  1441548],
 'EB-31-32': [['EB-25-26-27-29-33'],
  ['EB-25-26-27-29-33'],
  0.235688,
  56139,
  ['186', '211', '210', '208', '209', '190'],
  ['31', '32'],
  719384],
 'EB-25-26-27-29-33': [['EB-28-30', 'EB-31-32'],
  ['EB-28-30', 'EB-31-32', 'EB-28-30', 'EB-31-32', '34'],
  0.580807,
  569596,
  ['18',
   '17',
   '15',
   '16',
   '1',
   '4',
   '30',
   '24',
   '88',
   '95',
   '96',
   '97',
   '101',
   '102',
   '103',
   '87'],
  ['25', '26', '27', '29', '33'],
  2880404]}

In [52]:
for r in final_result[best_category[0]['index'][0]]:
    print(final_result[best_category[0]['index'][0]][r])

[['EB-25-26-27-29-33'], ['35'], 1.182939, 555294969, []]
[['34'], [], 0.407706, 1, []]
[['EB-25-26-27-29-33'], ['EB-25-26-27-29-33'], 0.23311700000000002, 135789, ['150', '149', '147', '148', '125', '128'], ['28', '30'], 1441548]
[['EB-25-26-27-29-33'], ['EB-25-26-27-29-33'], 0.235688, 56139, ['186', '211', '210', '208', '209', '190'], ['31', '32'], 719384]
[['EB-28-30', 'EB-31-32'], ['EB-28-30', 'EB-31-32', 'EB-28-30', 'EB-31-32', '34'], 0.580807, 569596, ['18', '17', '15', '16', '1', '4', '30', '24', '88', '95', '96', '97', '101', '102', '103', '87'], ['25', '26', '27', '29', '33'], 2880404]
